In [4]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("amazon_polarity")  # Replace with actual dataset name if needed

def preprocess_function(examples):
    # Normalize text by converting to lowercase and removing special characters
    # Access the correct column name, e.g., 'content'
    examples['content'] = [text.lower() for text in examples['content']]
    return examples

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Map:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # 3 sentiment labels: Positive, Negative, Neutral

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["content"], truncation=True, padding=True)  # Changed "text" to "content"

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3600000 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

In [ ]:
trainer.train()

In [ ]:
results = trainer.evaluate()
print("Test Accuracy:", results['eval_accuracy'])